<a href="https://colab.research.google.com/github/Kethanvr/Reag-VecotrDB/blob/main/Final_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Install all required packages
!pip install -q pymongo sentence-transformers google-generativeai langchain langchain-google-genai langchain-community pypdf python-docx openpyxl pandas unstructured pillow langchain-text-splitters

In [39]:
import os
import hashlib
import numpy as np
from typing import List, Dict, Any
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pymongo import MongoClient
from google.colab import userdata

# LangChain imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    TextLoader,
    UnstructuredExcelLoader,
    CSVLoader,
    UnstructuredMarkdownLoader,
    UnstructuredHTMLLoader
)
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI

# For file uploads
from google.colab import files
import tempfile

In [40]:
import os
import hashlib
import numpy as np
from typing import List, Dict, Any
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pymongo import MongoClient
from google.colab import userdata

# LangChain imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    TextLoader,
    UnstructuredExcelLoader,
    CSVLoader,
    UnstructuredMarkdownLoader,
    UnstructuredHTMLLoader
)
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI

# For file uploads
from google.colab import files
import tempfile

In [53]:
# Get API keys from Colab Secrets
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
MONGODB_URI = userdata.get('MONGODB_URI')

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Configuration loaded successfully!")

✅ Configuration loaded successfully!


In [52]:
class AdvancedRAGSystem:
    def __init__(self, mongodb_uri: str, db_name: str = "rag", collection_name: str = "documents"):
        """Initialize Advanced RAG system with multi-format support"""

        # 1. Load BGE embedding model (1024 dimensions)
        print("🔄 Loading BGE-large model (1024D)...")
        self.embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')
        self.embedding_dim = 1024

        # 2. Initialize MongoDB connection
        print("🔄 Connecting to MongoDB Atlas...")
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

        # 3. Initialize Gemini model via LangChain
        print("🔄 Initializing Gemini model...")
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite",
            google_api_key=GEMINI_API_KEY,
            temperature=0.3
        )

        # 4. Initialize text splitter (LangChain)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )

        print("✅ Advanced RAG System initialized successfully!")

    def load_file(self, file_path: str) -> List[Document]:
        """Load any file format using LangChain loaders"""

        file_extension = os.path.splitext(file_path)[1].lower()

        print(f"📄 Loading {file_extension} file...")

        try:
            if file_extension == '.pdf':
                loader = PyPDFLoader(file_path)
            elif file_extension in ['.docx', '.doc']:
                loader = Docx2txtLoader(file_path)
            elif file_extension == '.txt':
                loader = TextLoader(file_path)
            elif file_extension in ['.xlsx', '.xls']:
                loader = UnstructuredExcelLoader(file_path)
            elif file_extension == '.csv':
                loader = CSVLoader(file_path)
            elif file_extension == '.md':
                loader = UnstructuredMarkdownLoader(file_path)
            elif file_extension in ['.html', '.htm']:
                loader = UnstructuredHTMLLoader(file_path)
            else:
                print(f"⚠️ Unsupported file format: {file_extension}")
                return []

            documents = loader.load()
            print(f"✅ Loaded {len(documents)} page(s)/section(s)")
            return documents

        except Exception as e:
            print(f"❌ Error loading file: {e}")
            return []

    def chunk_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks using LangChain"""
        print("🔄 Splitting documents into chunks...")
        chunks = self.text_splitter.split_documents(documents)
        print(f"✅ Created {len(chunks)} chunks")
        return chunks

    def generate_hash(self, text: str) -> str:
        """Generate SHA-256 hash for deduplication"""
        return hashlib.sha256(text.encode()).hexdigest()

    def ingest_file(self, file_path: str, metadata: Dict[str, Any] = None):
        """Ingest any file: load, chunk, embed, and store"""
        if metadata is None:
            metadata = {}

        # Step 1: Load the file
        documents = self.load_file(file_path)
        if not documents:
            print("❌ No documents loaded. Aborting ingestion.")
            return

        # Step 2: Chunk the documents
        chunks = self.chunk_documents(documents)

        # Step 3: Generate embeddings and store
        print("🔄 Generating embeddings and storing in MongoDB...")
        stored_count = 0
        skipped_count = 0

        for i, chunk in enumerate(chunks):
            # Generate embedding (1024D vector)
            embedding = self.embedding_model.encode(chunk.page_content).tolist()

            # Generate dedup hash
            chunk_hash = self.generate_hash(chunk.page_content)

            # Check if chunk already exists (deduplication)
            existing = self.collection.find_one({"hash": chunk_hash})
            if existing:
                skipped_count += 1
                continue

            # Merge metadata
            chunk_metadata = {**metadata, **chunk.metadata, "chunk_index": i}

            # Store in MongoDB
            document = {
                "text": chunk.page_content,
                "embedding": embedding,
                "hash": chunk_hash,
                "metadata": chunk_metadata
            }

            self.collection.insert_one(document)
            stored_count += 1

            if (i + 1) % 10 == 0:
                print(f"  ✓ Processed {i + 1}/{len(chunks)} chunks...")

        print(f"✅ Ingestion complete! Stored: {stored_count}, Skipped (duplicates): {skipped_count}")

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Perform semantic search using vector similarity"""
        # Generate query embedding
        query_embedding = self.embedding_model.encode(query).tolist()

        # MongoDB Vector Search Pipeline
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": query_embedding,
                    "numCandidates": top_k * 10,
                    "limit": top_k
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "text": 1,
                    "metadata": 1,
                    "score": {"$meta": "vectorSearchScore"}
                }
            }
        ]

        try:
            results = list(self.collection.aggregate(pipeline))
            return results
        except Exception as e:
            print(f"⚠️ Search error: {e}")
            print("💡 Make sure vector search index 'vector_index' is created!")
            return []

    def generate_answer(self, query: str, top_k: int = 3) -> Dict[str, Any]:
        """Generate answer using RAG approach with LangChain"""
        # Step 1: Retrieve relevant context
        print(f"🔍 Searching for relevant context...")
        search_results = self.search(query, top_k=top_k)

        if not search_results:
            return {
                "answer": "⚠️ I couldn't find relevant information. Please ensure:\n1. Documents are uploaded\n2. Vector search index is created in MongoDB Atlas",
                "sources": [],
                "context_used": [],
                "num_sources": 0
            }

        # Step 2: Prepare context
        context_chunks = [result['text'] for result in search_results]
        context = "\n\n".join([f"[Source {i+1}]\n{chunk}" for i, chunk in enumerate(context_chunks)])

        # Step 3: Create prompt
        prompt = f"""You are a helpful AI assistant. Answer the question based ONLY on the provided context.
If the context doesn't contain enough information, clearly state that.
Do NOT make up information or use external knowledge.

Context:
{context}

Question: {query}

Provide a clear, concise answer:"""

        # Step 4: Generate answer using Gemini via LangChain
        print("🤖 Generating answer with Gemini...")
        try:
            response = self.llm.invoke(prompt)
            answer = response.content
        except Exception as e:
            print(f"❌ Error generating answer: {e}")
            answer = "Sorry, I encountered an error while generating the answer."

        return {
            "answer": answer,
            "sources": search_results,
            "context_used": context_chunks,
            "num_sources": len(search_results)
        }

print("✅ AdvancedRAGSystem class defined successfully!")

✅ AdvancedRAGSystem class defined successfully!


In [54]:
print("\n" + "="*80)
print("🚀 INITIALIZING ADVANCED RAG SYSTEM")
print("="*80)

rag = AdvancedRAGSystem(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)


🚀 INITIALIZING ADVANCED RAG SYSTEM
🔄 Loading BGE-large model (1024D)...
🔄 Connecting to MongoDB Atlas...
🔄 Initializing Gemini model...
✅ Advanced RAG System initialized successfully!


In [55]:
print("\n" + "="*80)
print("📤 UPLOAD YOUR DOCUMENTS")
print("="*80)
print("""
Supported formats:
✓ PDF (.pdf)
✓ Word (.docx, .doc)
✓ Text (.txt)
✓ Excel (.xlsx, .xls)
✓ CSV (.csv)
✓ Markdown (.md)
✓ HTML (.html, .htm)

🔒 Deduplication: Enabled (SHA-256 hashing)
   → Duplicate chunks will be automatically skipped
""")

# Upload files
uploaded = files.upload()

# Track statistics
total_stored = 0
total_skipped = 0

# Process each uploaded file
for filename, file_content in uploaded.items():
    print(f"\n{'='*60}")
    print(f"📄 Processing: {filename}")
    print(f"{'='*60}")

    # Save to temporary file
    temp_path = f"/tmp/{filename}"
    with open(temp_path, 'wb') as f:
        f.write(file_content)

    # Ingest the file
    rag.ingest_file(
        file_path=temp_path,
        metadata={
            "source": filename,
            "file_type": os.path.splitext(filename)[1]
        }
    )

    # Clean up
    os.remove(temp_path)
    print(f"✅ {filename} processed\n")

print("\n" + "="*80)
print("📊 INGESTION SUMMARY")
print("="*80)
print(f"""
Total files processed: {len(uploaded)}
✅ Stored new chunks: Check MongoDB stats
🔒 Skipped duplicates: Check individual file outputs above
""")


📤 UPLOAD YOUR DOCUMENTS

Supported formats:
✓ PDF (.pdf)
✓ Word (.docx, .doc)
✓ Text (.txt)
✓ Excel (.xlsx, .xls)
✓ CSV (.csv)
✓ Markdown (.md)
✓ HTML (.html, .htm)



Saving Kethan VR A Comprehensive.md to Kethan VR A Comprehensive (1).md

📄 Processing: Kethan VR A Comprehensive (1).md
📄 Loading .md file...
✅ Loaded 1 page(s)/section(s)
🔄 Splitting documents into chunks...
✅ Created 24 chunks
🔄 Generating embeddings and storing in MongoDB...
✅ Ingestion complete! Stored: 0, Skipped (duplicates): 24

✅ All files processed successfully!


In [57]:
# Run this ONCE to create hash index
print("🔧 Creating hash index for faster deduplication...")
rag.collection.create_index("hash", unique=True)
print("✅ Hash index created!")

🔧 Creating hash index for faster deduplication...
✅ Hash index created!


In [58]:
# First install rich for beautiful formatting
!pip install -q rich


In [59]:

# Then use this enhanced version:

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.table import Table
from rich import box

console = Console()

print("\n" + "="*80)
console.print("💬 [bold cyan]INTERACTIVE Q&A SYSTEM[/bold cyan]")
print("="*80)
console.print("[yellow]Ask questions about your uploaded documents![/yellow]")
console.print("[dim]Type 'exit' or 'quit' to stop.[/dim]\n")

while True:
    # Get user question
    question = console.input("[bold green]❓ Your question:[/bold green] ").strip()

    # Exit condition
    if question.lower() in ['exit', 'quit', 'q', '']:
        console.print("\n[bold]👋 Goodbye![/bold]")
        break

    console.print("\n" + "="*80)

    # Generate answer
    result = rag.generate_answer(question, top_k=3)

    # ========================================================================
    # FORMATTED ANSWER OUTPUT WITH RICH
    # ========================================================================

    answer = result.get('answer', 'No answer generated')

    # Display answer in a panel
    console.print(Panel(
        Markdown(answer),
        title="🤖 Answer",
        border_style="cyan",
        box=box.ROUNDED
    ))

    # ========================================================================
    # SOURCE INFORMATION TABLE
    # ========================================================================

    num_sources = result.get('num_sources', 0)
    sources = result.get('sources', [])

    if num_sources > 0:
        # Create sources table
        table = Table(title=f"📚 Sources ({num_sources} found)", box=box.ROUNDED)
        table.add_column("#", style="cyan", width=4)
        table.add_column("File", style="green")
        table.add_column("Page/Section", style="yellow")
        table.add_column("Relevance", style="magenta")

        for i, source in enumerate(sources, 1):
            metadata = source.get('metadata', {})
            score = source.get('score', 0)

            table.add_row(
                str(i),
                metadata.get('source', 'Unknown'),
                str(metadata.get('page', 'N/A')),
                f"{score:.4f}"
            )

        console.print(table)

        # ====================================================================
        # OPTION TO VIEW FULL CONTEXT
        # ====================================================================

        console.print("\n[bold]Options:[/bold]")
        console.print("  [1] Show all source context")
        console.print("  [2] Show specific source")
        console.print("  [3] Continue to next question")

        choice = console.input("\n[bold yellow]Select option (1/2/3):[/bold yellow] ").strip()

        if choice == '1':
            # Show all context
            context_used = result.get('context_used', [])
            for i, context in enumerate(context_used, 1):
                console.print(Panel(
                    context,
                    title=f"📖 Source {i} - Full Context",
                    border_style="blue",
                    box=box.ROUNDED
                ))

        elif choice == '2':
            # Show specific source
            source_num = console.input(f"\n[bold]Which source? (1-{num_sources}):[/bold] ").strip()
            try:
                idx = int(source_num) - 1
                if 0 <= idx < num_sources:
                    context_used = result.get('context_used', [])
                    console.print(Panel(
                        context_used[idx],
                        title=f"📖 Source {source_num} - Full Context",
                        border_style="green",
                        box=box.ROUNDED
                    ))
                else:
                    console.print("[red]⚠️ Invalid source number[/red]")
            except ValueError:
                console.print("[red]⚠️ Please enter a valid number[/red]")

    else:
        console.print(Panel(
            "[yellow]Make sure:\n1. Documents are uploaded\n2. Vector search index is created\n3. Question relates to uploaded content[/yellow]",
            title="⚠️ No Sources Found",
            border_style="red",
            box=box.ROUNDED
        ))

    console.print("\n" + "="*80 + "\n")

💬 INTERACTIVE Q&A SYSTEM

Ask questions about your uploaded documents!

Type 'exit' or 'quit' to stop.

❓ Your question:

who is kethan vr and give is social media links


...

╭────────────────────────────────────────── 📖 Source 2 - Full Context ───────────────────────────────────────────╮
│ Kethan VR: A Comprehensive Research Profile of an Emerging Tech Innovator                                       │
│                                                                                                                 │
│ Author: AI Research Analysis Date: January 18, 2026 Location: Bangalore, Karnataka, India Status: Early-Stage   │
│ Developer & Engineering Student | AI-First Builder                                                              │
│                                                                                                                 │
│ Executive Summary                                                                                               │
│                                                                                                                 │
│ Kethan VR represents a new breed of Gen Z tech innovators in India—the kind rewriting the startup ecosystem     │
│ with bold ideas, shipped products, and an authentic online presence. At 19-20, he's already built a compelling  │
│ fullstack portfolio, shipped multiple AI-integrated projects, and positioned himself at the intersection of web │
│ development, AI/SaaS, and India's thriving startup culture.                                                     │
│                                                                                                                 │
│ Unlike the "resume polishers," Kethan is shipping real stuff: CoCreateAI (an AI prompting platform), MediScan   │
│ (AI health assistant), and a portfolio site that actually converts. He's active across GitHub, LinkedIn,        │
│ YouTube, and Twitter—building a genuine personal brand without the hype.                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── 📖 Source 3 - Full Context ───────────────────────────────────────────╮
│ Startup India: Government grants and mentorship programs                                                        │
│                                                                                                                 │
│ Product Hunt: Launch platform for shipping products to 1M+ audience                                             │
│                                                                                                                 │
│ Devfolio: Hackathon platform (Kethan likely already active here)                                                │
│                                                                                                                 │
│ Profile Prepared: January 18, 2026 Next Review: July 18, 2026 (6-month check-in)                                │
│                                                                                                                 │
│ This profile is intended as a research document analyzing Kethan VR's position in India's Gen Z startup         │
│ ecosystem, his technical positioning, and the macro opportunity he's entering. It's based on public portfolio,  │
│ LinkedIn, GitHub, and broader market trends in Indian startups and AI-first SaaS.                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

================================================================================

❓ Your question:

KeyboardInterrupt: Interrupted by user

In [60]:
!pip install -q duckduckgo-search beautifulsoup4 requests Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.4 MB/s eta 0:00:00


In [61]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from duckduckgo_search import DDGS
from typing import List, Dict, Any, Optional
import json

class AdvancedRAGWithMemoryVisionWeb:
    def __init__(self, mongodb_uri: str, db_name: str = "rag", collection_name: str = "documents"):
        """Initialize Advanced RAG with Memory, Vision, and Web capabilities"""

        # 1. Load BGE embedding model
        print("🔄 Loading BGE-large model (1024D)...")
        self.embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')
        self.embedding_dim = 1024

        # 2. Initialize MongoDB connection
        print("🔄 Connecting to MongoDB Atlas...")
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]
        self.memory_collection = self.db["conversation_memory"]
        self.important_info_collection = self.db["important_info"]

        # 3. Initialize Gemini 2.5 Flash-Lite with vision support
        print("🔄 Initializing Gemini 2.5 Flash-Lite...")
        genai.configure(api_key=GEMINI_API_KEY)
        self.llm = genai.GenerativeModel('gemini-2.5-flash-lite')

        # 4. Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )

        # 5. Initialize conversation memory (in-session)
        self.conversation_history = []
        self.session_id = None

        print("✅ Advanced RAG System with Memory, Vision & Web initialized!")

    # ========================================================================
    # MEMORY FUNCTIONS
    # ========================================================================

    def start_new_session(self) -> str:
        """Start a new conversation session"""
        import uuid
        self.session_id = str(uuid.uuid4())
        self.conversation_history = []
        print(f"🆕 New session started: {self.session_id[:8]}...")
        return self.session_id

    def add_to_memory(self, role: str, content: str, metadata: Dict = None):
        """Add message to conversation memory"""
        message = {
            "role": role,
            "content": content,
            "timestamp": hashlib.sha256(str(np.random.random()).encode()).hexdigest()[:16]
        }

        if metadata:
            message["metadata"] = metadata

        self.conversation_history.append(message)

        # Store in MongoDB for persistence
        memory_doc = {
            "session_id": self.session_id,
            "role": role,
            "content": content,
            "metadata": metadata,
            "timestamp": message["timestamp"]
        }
        self.memory_collection.insert_one(memory_doc)

    def get_conversation_context(self, last_n: int = 5) -> str:
        """Get recent conversation history as context"""
        recent = self.conversation_history[-last_n:]
        context = "\n".join([f"{msg['role']}: {msg['content']}" for msg in recent])
        return context

    def save_important_info(self, info: str, category: str = "general"):
        """Save important information for long-term memory"""
        doc = {
            "info": info,
            "category": category,
            "embedding": self.embedding_model.encode(info).tolist(),
            "timestamp": hashlib.sha256(str(np.random.random()).encode()).hexdigest()[:16]
        }
        self.important_info_collection.insert_one(doc)
        print(f"💾 Saved important info: {info[:50]}...")

    def recall_important_info(self, query: str, top_k: int = 3) -> List[str]:
        """Retrieve important information from long-term memory"""
        query_embedding = self.embedding_model.encode(query).tolist()

        pipeline = [
            {
                "$vectorSearch": {
                    "index": "memory_index",
                    "path": "embedding",
                    "queryVector": query_embedding,
                    "numCandidates": top_k * 10,
                    "limit": top_k
                }
            },
            {"$project": {"_id": 0, "info": 1}}
        ]

        try:
            results = list(self.important_info_collection.aggregate(pipeline))
            return [r['info'] for r in results]
        except:
            return []

    # ========================================================================
    # VISION FUNCTIONS
    # ========================================================================

    def analyze_image_from_path(self, image_path: str, question: str = "Describe this image in detail") -> str:
        """Analyze image from file path"""
        try:
            with open(image_path, 'rb') as f:
                image_data = f.read()

            response = self.llm.generate_content([
                question,
                {"mime_type": "image/jpeg", "data": image_data}
            ])

            return response.text
        except Exception as e:
            return f"Error analyzing image: {e}"

    def analyze_image_from_url(self, image_url: str, question: str = "Describe this image") -> str:
        """Analyze image from URL"""
        try:
            response = requests.get(image_url)
            image_data = response.content

            response = self.llm.generate_content([
                question,
                {"mime_type": "image/jpeg", "data": image_data}
            ])

            return response.text
        except Exception as e:
            return f"Error analyzing image: {e}"

    def analyze_uploaded_image(self, uploaded_file, question: str = "Describe this image") -> str:
        """Analyze image uploaded in Colab"""
        try:
            # For Colab file upload
            image_data = uploaded_file[list(uploaded_file.keys())[0]]

            response = self.llm.generate_content([
                question,
                {"mime_type": "image/jpeg", "data": image_data}
            ])

            return response.text
        except Exception as e:
            return f"Error analyzing image: {e}"

    # ========================================================================
    # WEB SCRAPING FUNCTIONS
    # ========================================================================

    def web_search(self, query: str, max_results: int = 5) -> List[Dict]:
        """Search the web using DuckDuckGo"""
        try:
            print(f"🔍 Searching web for: {query}")
            ddgs = DDGS()
            results = list(ddgs.text(query, max_results=max_results))
            print(f"✅ Found {len(results)} results")
            return results
        except Exception as e:
            print(f"❌ Web search error: {e}")
            return []

    def scrape_webpage(self, url: str) -> str:
        """Scrape and extract text from a webpage"""
        try:
            print(f"🌐 Scraping: {url}")
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()

            # Get text
            text = soup.get_text()

            # Clean up text
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = ' '.join(chunk for chunk in chunks if chunk)

            print(f"✅ Scraped {len(text)} characters")
            return text[:5000]  # Limit to 5000 chars
        except Exception as e:
            print(f"❌ Scraping error: {e}")
            return ""

    def web_rag(self, query: str, num_results: int = 3) -> str:
        """Search web, scrape pages, and answer question"""
        # Search web
        search_results = self.web_search(query, max_results=num_results)

        if not search_results:
            return "No web results found."

        # Scrape top results
        scraped_content = []
        for result in search_results[:num_results]:
            url = result.get('href', '')
            title = result.get('title', '')

            if url:
                content = self.scrape_webpage(url)
                if content:
                    scraped_content.append({
                        'url': url,
                        'title': title,
                        'content': content
                    })

        # Generate answer from scraped content
        if not scraped_content:
            return "Could not scrape any content from search results."

        context = "\n\n".join([
            f"Source: {item['title']}\nURL: {item['url']}\nContent: {item['content'][:1000]}"
            for item in scraped_content
        ])

        prompt = f"""Based on the following web search results, answer the question.

Web Content:
{context}

Question: {query}

Answer (include source URLs):"""

        response = self.llm.generate_content(prompt)
        return response.text

    # ========================================================================
    # ENHANCED GENERATE ANSWER WITH MEMORY
    # ========================================================================

    def generate_answer_with_memory(self, query: str, use_web: bool = False, top_k: int = 3) -> Dict[str, Any]:
        """Generate answer using RAG + Memory + optionally Web"""

        # Add query to conversation memory
        self.add_to_memory("user", query)

        # Get conversation context
        conversation_context = self.get_conversation_context(last_n=5)

        # Recall important info
        recalled_info = self.recall_important_info(query, top_k=2)
        recalled_text = "\n".join(recalled_info) if recalled_info else ""

        # Search documents
        print(f"🔍 Searching documents...")
        search_results = self.search(query, top_k=top_k)

        # Optionally search web
        web_context = ""
        if use_web:
            print(f"🌐 Searching web...")
            web_context = self.web_rag(query, num_results=2)

        # Prepare context
        doc_context = ""
        if search_results:
            context_chunks = [result['text'] for result in search_results]
            doc_context = "\n\n".join([f"[Doc {i+1}]\n{chunk}" for i, chunk in enumerate(context_chunks)])

        # Create enhanced prompt with memory
        prompt = f"""You are a helpful AI assistant with memory of our conversation.

Conversation History:
{conversation_context}

Recalled Important Information:
{recalled_text}

Document Context:
{doc_context}

Web Context:
{web_context}

Current Question: {query}

Instructions:
- Answer based on all available context
- Reference previous conversation if relevant
- If using web info, cite sources
- Be conversational and remember our chat history

Answer:"""

        # Generate answer
        print("🤖 Generating answer...")
        try:
            response = self.llm.generate_content(prompt)
            answer = response.text
        except Exception as e:
            answer = f"Error generating answer: {e}"

        # Add answer to memory
        self.add_to_memory("assistant", answer)

        # Check if answer contains important info to save
        if "important" in query.lower() or "remember" in query.lower():
            self.save_important_info(answer, category="user_marked")

        return {
            "answer": answer,
            "sources": search_results,
            "web_used": use_web,
            "conversation_context": conversation_context,
            "recalled_info": recalled_info,
            "num_sources": len(search_results)
        }

    # ========================================================================
    # EXISTING FUNCTIONS (keep your ingest_file, search, etc.)
    # ========================================================================

    def load_file(self, file_path: str) -> List[Document]:
        """Load any file format using LangChain loaders"""
        file_extension = os.path.splitext(file_path)[1].lower()
        print(f"📄 Loading {file_extension} file...")

        try:
            if file_extension == '.pdf':
                loader = PyPDFLoader(file_path)
            elif file_extension in ['.docx', '.doc']:
                loader = Docx2txtLoader(file_path)
            elif file_extension == '.txt':
                loader = TextLoader(file_path)
            elif file_extension in ['.xlsx', '.xls']:
                loader = UnstructuredExcelLoader(file_path)
            elif file_extension == '.csv':
                loader = CSVLoader(file_path)
            elif file_extension == '.md':
                loader = UnstructuredMarkdownLoader(file_path)
            elif file_extension in ['.html', '.htm']:
                loader = UnstructuredHTMLLoader(file_path)
            else:
                print(f"⚠️ Unsupported file format: {file_extension}")
                return []

            documents = loader.load()
            print(f"✅ Loaded {len(documents)} page(s)/section(s)")
            return documents
        except Exception as e:
            print(f"❌ Error loading file: {e}")
            return []

    def chunk_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks"""
        print("🔄 Splitting documents...")
        chunks = self.text_splitter.split_documents(documents)
        print(f"✅ Created {len(chunks)} chunks")
        return chunks

    def generate_hash(self, text: str) -> str:
        """Generate SHA-256 hash"""
        return hashlib.sha256(text.encode()).hexdigest()

    def ingest_file(self, file_path: str, metadata: Dict[str, Any] = None):
        """Ingest file with deduplication"""
        if metadata is None:
            metadata = {}

        documents = self.load_file(file_path)
        if not documents:
            return

        chunks = self.chunk_documents(documents)

        print("🔄 Generating embeddings and storing...")
        stored = 0
        skipped = 0

        for i, chunk in enumerate(chunks):
            embedding = self.embedding_model.encode(chunk.page_content).tolist()
            chunk_hash = self.generate_hash(chunk.page_content)

            if self.collection.find_one({"hash": chunk_hash}):
                skipped += 1
                continue

            doc = {
                "text": chunk.page_content,
                "embedding": embedding,
                "hash": chunk_hash,
                "metadata": {**metadata, **chunk.metadata, "chunk_index": i}
            }

            self.collection.insert_one(doc)
            stored += 1

            if (i + 1) % 10 == 0:
                print(f"  ✓ {i + 1}/{len(chunks)} chunks...")

        print(f"✅ Stored: {stored}, Skipped: {skipped}")

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Vector search"""
        query_embedding = self.embedding_model.encode(query).tolist()

        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": query_embedding,
                    "numCandidates": top_k * 10,
                    "limit": top_k
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "text": 1,
                    "metadata": 1,
                    "score": {"$meta": "vectorSearchScore"}
                }
            }
        ]

        try:
            results = list(self.collection.aggregate(pipeline))
            return results
        except Exception as e:
            print(f"⚠️ Search error: {e}")
            return []

print("✅ AdvancedRAGWithMemoryVisionWeb class ready!")

✅ AdvancedRAGWithMemoryVisionWeb class ready!


In [62]:
print("\n" + "="*80)
print("🚀 INITIALIZING ENHANCED RAG SYSTEM")
print("="*80)

rag = AdvancedRAGWithMemoryVisionWeb(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)

# Start a new conversation session
rag.start_new_session()


🚀 INITIALIZING ENHANCED RAG SYSTEM
🔄 Loading BGE-large model (1024D)...
🔄 Connecting to MongoDB Atlas...
🔄 Initializing Gemini 2.5 Flash-Lite...
✅ Advanced RAG System with Memory, Vision & Web initialized!
🆕 New session started: bbbf3dfc...


'bbbf3dfc-ac74-417d-8936-d9ffd3e0233b'

In [63]:
print("\n" + "="*80)
print("💬 ENHANCED RAG SYSTEM - Memory + Vision + Web")
print("="*80)
print("""
Commands:
- Just ask a question (uses documents + memory)
- 'web: your question' (search web + scrape)
- 'image' (analyze an image)
- 'remember: info' (save to long-term memory)
- 'recall: topic' (retrieve from memory)
- 'exit' (quit)
""")

while True:
    user_input = input("\n❓ You: ").strip()

    if user_input.lower() in ['exit', 'quit', 'q']:
        print("👋 Goodbye!")
        break

    print("\n" + "-"*80)

    # Handle different commands
    if user_input.startswith('web:'):
        # Web search mode
        query = user_input[4:].strip()
        result = rag.generate_answer_with_memory(query, use_web=True)
        print(f"🤖 Answer:\n{result['answer']}\n")
        print(f"🌐 Web search: Enabled")
        print(f"📚 Documents used: {result['num_sources']}")

    elif user_input.lower() == 'image':
        # Image analysis mode
        print("📤 Upload an image...")
        uploaded = files.upload()

        if uploaded:
            question = input("❓ What do you want to know about this image? ")
            answer = rag.analyze_uploaded_image(uploaded, question)
            print(f"\n🤖 Image Analysis:\n{answer}\n")

            # Add to memory
            rag.add_to_memory("user", f"Analyzed image: {question}")
            rag.add_to_memory("assistant", answer)

    elif user_input.startswith('remember:'):
        # Save to long-term memory
        info = user_input[9:].strip()
        rag.save_important_info(info)
        print("💾 Saved to long-term memory!")

    elif user_input.startswith('recall:'):
        # Recall from memory
        topic = user_input[7:].strip()
        recalled = rag.recall_important_info(topic)
        if recalled:
            print("🧠 Recalled from memory:")
            for i, info in enumerate(recalled, 1):
                print(f"\n{i}. {info}")
        else:
            print("⚠️ Nothing found in memory")

    else:
        # Normal RAG with memory
        result = rag.generate_answer_with_memory(user_input, use_web=False)
        print(f"🤖 Answer:\n{result['answer']}\n")

        if result['num_sources'] > 0:
            print(f"📚 Sources: {result['num_sources']}")

            show = input("View sources? (y/n): ").strip().lower()
            if show == 'y':
                for i, source in enumerate(result['sources'], 1):
                    print(f"\n[Source {i}]")
                    print(f"File: {source['metadata'].get('source', 'Unknown')}")
                    print(f"Score: {source.get('score', 0):.4f}")

    print("-"*80)


💬 ENHANCED RAG SYSTEM - Memory + Vision + Web

Commands:
- Just ask a question (uses documents + memory)
- 'web: your question' (search web + scrape)
- 'image' (analyze an image)
- 'remember: info' (save to long-term memory)
- 'recall: topic' (retrieve from memory)
- 'exit' (quit)


❓ You: hi

--------------------------------------------------------------------------------
🔍 Searching documents...
🤖 Generating answer...
🤖 Answer:
Hi there! How can I help you today?

📚 Sources: 3
View sources? (y/n): y

[Source 1]
File: /tmp/Kethan VR A Comprehensive.md
Score: 0.7573

[Source 2]
File: /tmp/Kethan VR A Comprehensive.md
Score: 0.7533

[Source 3]
File: /tmp/Kethan VR A Comprehensive.md
Score: 0.7503
--------------------------------------------------------------------------------



KeyboardInterrupt

